# Dependências

In [106]:
%pip install tensorflow
%pip install opencv-python
%pip install mediapipe
%pip install numpy
%pip install cv2
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


Note: you may need to restart the kernel to use updated packages.


# Imports e Funções

In [2]:
import cv2
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt
import time
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.regularizers import l1, l2, l1_l2

mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, modelh, modelp):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    resultsh = modelh.process(image)                 # Make prediction
    resultsp = modelp.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, resultsh, resultsp

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.multi_hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw left hand connections

def draw_styled_landmarks(image, resultsh, resultsp):
    # Desenhar landmarks da pose
    if resultsp.pose_landmarks:
        mp_drawing.draw_landmarks(image, resultsp.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2), # Landmark color
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)   # Connection color
                                 )
    
    # Desenhar landmarks das mãos
    if resultsh.multi_hand_landmarks:
        for hand_landmarks in resultsh.multi_hand_landmarks: # BURRO!
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                     mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4), # Landmark color (e.g., blue for hands)
                                     mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2)   # Connection color (e.g., red for hands)
                                     )
def extract_keypoints(resultsh, resultsp):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in resultsp.pose_landmarks.landmark]).flatten() if resultsp.pose_landmarks else np.zeros(132)
    rh = lh = np.zeros(63)
    if resultsh.multi_hand_landmarks:
        
        # Itera sobre cada mão detectada
        for idx, hand_landmarks in enumerate(resultsh.multi_hand_landmarks):
            
            # Verifica diretamente se a mão atual é a Direita
            if resultsh.multi_handedness[idx].classification[0].label == "Right":
                print(f"Mão Direita detectada no índice {idx}.")
                # Lógica para extrair os landmarks da mão direita
                rh = np.array([[res.x, res.y, res.z] for res in hand_landmarks.landmark]).flatten() if resultsh.multi_hand_landmarks else np.zeros(63)
    
            # Verifica diretamente se a mão atual é a Esquerda
            elif resultsh.multi_handedness[idx].classification[0].label == "Left":
                print(f"Mão Esquerda detectada no índice {idx}.")
                # landmarks da mão esquerda
                lh = np.array([[res.x, res.y, res.z] for res in hand_landmarks.landmark]).flatten() if resultsh.multi_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, lh, rh])

# Abrir Câmera

In [60]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
    
            # Read feed
            ret, frame = cap.read()
            frame = cv2.flip(frame,1)
            # Make detections
            image, resultsh, resultsp = mediapipe_detection(frame, hands, pose)
            
            # Draw landmarks
            draw_styled_landmarks(image, resultsh, resultsp)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)
    
            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

## Extração de keypoints

## Definição de Sinais (actions)

In [90]:

DATA_PATH = os.path.join('MP_Data') 

actions = np.array(['Oi', 'Obrigado','Bom','Dia','Tarde','Noite','Por favor'])
#'Oi', 'Obrigado','Bom','Dia','Tarde','Noite','Dinheiro','Por favor'
no_sequences = 30 #Videos

sequence_length = 30 #frames

## Criação de pastas (se não existirem)

In [25]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Capturando Keypoints para Treinamento

In [ ]:
quit = False
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        for action in actions:
            for sequence in range(no_sequences):
                for frame_num in range(sequence_length):
    
                    # Read feed
                    ret, frame = cap.read()
                    frame = cv2.flip(frame,1)
                    # Make detections
                    image, resultsh, resultsp = mediapipe_detection(frame, hands, pose)
                    
                    # Draw landmarks
                    draw_styled_landmarks(image, resultsh, resultsp)
                    # Wait and texts
                    if frame_num == 0:
                        cv2.putText(image, '{}'.format(action), (120,200),cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                        cv2.putText(image, 'SINAL {} NUMERO {}'.format(action,sequence), (40,30),cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)
                        cv2.waitKey(2000)
                        
                    else:
                        cv2.putText(image, 'SINAL {} NUMERO {}'.format(action,sequence), (15,12),cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)
                    keypoints = extract_keypoints(resultsh, resultsp)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)


                    
                    
            
                    # Break gracefully
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        quit = True
                        break
                if quit:
                    break
            if quit:
                    break
cap.release()
cv2.destroyAllWindows()

Mão Esquerda detectada no índice 0.
Mão Esquerda detectada no índice 0.
Mão Esquerda detectada no índice 0.
Mão Esquerda detectada no índice 0.
Mão Esquerda detectada no índice 0.
Mão Esquerda detectada no índice 0.
Mão Esquerda detectada no índice 0.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão Esquerda detectada no índice 1.
Mão Direita detectada no índice 0.
Mão

# EM CASO DE EMERGÊNCIA EXECUtAR A CÉLULA

In [22]:
    cap.release()
    cv2.destroyAllWindows()

# PREPROCESSAMENTO E TREINO DE MODELO

In [92]:
label_map = {label:num for num, label in enumerate(actions)}
label_map
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test, = train_test_split(x,y,test_size=0.05)
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = models.Sequential()
forca = 0.00002
model.add(layers.Input(shape=(30,258)))
model.add(layers.LSTM(64, return_sequences=True, activation='relu', kernel_regularizer=l1_l2(l1=forca, l2=forca)))
model.add(layers.LSTM(128, return_sequences=True, activation='relu', kernel_regularizer=l1_l2(l1=forca, l2=forca)))
model.add(layers.LSTM(64, return_sequences=False, activation='relu', kernel_regularizer=l1_l2(l1=forca, l2=forca)))
model.add(layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=forca, l2=forca)))
model.add(layers.Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=forca, l2=forca)))
model.add(layers.Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(x_train,y_train, epochs=600, callbacks=[tb_callback])

In [46]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 30, 64)              │          82,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 30, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 12)                  │             396 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 712,646 (2.72 MB)

 Trainable params: 237,548 (927.92 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 475,098 (1.81 MB)

# SALVAR E CARREGAR O MODELO, RESPECTIVAMENTE

In [107]:
model.save('MODELO_GESTOS.keras')

In [10]:
model.load_weights('MODELO_GESTOS.keras')

In [13]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [14]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

IndexError: list index out of range

<Figure size 1800x1800 with 0 Axes>

In [105]:
cap = cv2.VideoCapture(0)
sequence = []
sentence = []
threshold = 0.90
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
    

            ret, frame = cap.read()
            frame = cv2.flip(frame,1)

            image, resultsh, resultsp = mediapipe_detection(frame, hands, pose)
            

            draw_styled_landmarks(image, resultsh, resultsp)


            keypoints = extract_keypoints(resultsh, resultsp)

            sequence.append(keypoints)
            sequence = sequence[-30:]
        
            if len(sequence) == 30:####################################################################################
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])    
                
                #3. Viz logic
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
    
                if len(sentence) > 5: 
                    sentence = sentence[-5:]
    
            
            cv2.rectangle(image, (0,0), (640, 40), (255, 255, 255), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)
    
            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            if 0xFF == ord('l'):
                cv2.rectangle(image, (0,0), (640, 40), (255, 255, 255), -1)
        cap.release()
        cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Dia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/s